# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [27]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [28]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [29]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
    #print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
    print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88']
['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88']
['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88']
['Barry Tuckwell/Ac

In [30]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [31]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [32]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cassproj 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [33]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('cassproj')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [34]:
## Create a table/data model that enables querying the music library data by session id and item in session
## The partition key is 'session_id'
## The cluster key is 'item_in_session'

#drop the table if exists, so that we can start afresh
table1_drop = "DROP TABLE IF EXISTS music_library_by_sessionid"

try:
    session.execute(table1_drop)
except Exception as e:
    print(e)
    
table1_create = "CREATE TABLE IF NOT EXISTS music_library_by_sessionid "
table1_create = table1_create + "(item_in_session int,session_id int,song_length decimal, artist text,song_name text, PRIMARY KEY(item_in_session,session_id))"
    
try:
    session.execute(table1_create)
except Exception as e:
    print(e)
                    

In [35]:
## Insert required data in appropriate columns in the order of 
## partition key, cluster key followed by other fields

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_by_sessionid (item_in_session,session_id,artist,song_name,song_length) "
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[3]), int(line[8]),line[0],line[9],float(line[5])))
            
            

#### Do a SELECT to verify that the data have been inserted into each table

In [36]:
## The following SELECT statement will fetch artist,song title and length of the songs for a given item in session and session id.

select1 = "SELECT artist,song_name,song_length FROM music_library_by_sessionid WHERE item_in_session = 4 AND session_id = 338"
try:
    rows = session.execute(select1)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song_name, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [40]:
## Create a table/data model that enables querying the music library data by user id and session id
## The partition key is 'user_id'
## The cluster key is 'session_id'

table2_drop = "DROP TABLE IF EXISTS music_library_by_user"

try:
    session.execute(table2_drop)
except Exception as e:
    print(e)
    
table2_create = "CREATE TABLE IF NOT EXISTS music_library_by_user "
table2_create = table2_create + "(user_id int,session_id int, item_in_session int, artist text, song_name text, first_name text, last_name text, PRIMARY KEY((user_id,session_id), item_in_session))"
    
try:
    session.execute(table2_create)
except Exception as e:
    print(e)


                    

In [41]:
## Insert required data in appropriate columns in the order of 
## partition key, cluster key followed by other fields

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_by_user (user_id,session_id,item_in_session,artist,song_name,first_name,last_name) "
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))
            

In [42]:
## The following SELECT statement will fetch name of artist, song (sorted by item in session) and user (first and last name) 
## for given user_id and session_id
## in this case for user_id = 10 and session_id = 182

select2 = "SELECT item_in_session,artist,song_name,first_name,last_name FROM music_library_by_user WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(select2)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song_name, row.first_name,row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [24]:
## Create a table/data model that enables querying the music library data by song name and user id
## The partition key is 'song_name'
## The cluster key is 'user_id'

table3_drop = "DROP TABLE IF EXISTS music_library_by_song"

try:
    session.execute(table3_drop)
except Exception as e:
    print(e)
    
table3_create = "CREATE TABLE IF NOT EXISTS music_library_by_song "
table3_create = table3_create + "(song_name text,user_id int, first_name text, last_name text, PRIMARY KEY(song_name,user_id))"
    
try:
    session.execute(table3_create)
except Exception as e:
    print(e)

                    

In [39]:
## Insert required data in appropriate columns in the order of 
## partition key, cluster key followed by other fields

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library_by_song (song_name,user_id,first_name,last_name) "
        query = query + "VALUES (%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]), line[1],line[4]))
            

In [26]:
## The Following SELECT statement will give list of all users that listened to a given 
## song name.  In this case, we are specifically looking for  users that
## listened to the song 'All Hands Against His Own'

select3 = "SELECT first_name, last_name FROM music_library_by_song WHERE song_name = 'All Hands Against His Own'"

try:
    rows = session.execute(select3);
except Exception as e:
    print(e)
for row in rows:
    print (row.first_name,row.last_name)    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [35]:
## Do the clean up by dropping all the tables

try:
    session.execute("DROP TABLE IF EXISTS music_library_by_sessionid")
    session.execute("DROP TABLE IF EXISTS music_library_by_user")
    session.execute("DROP TABLE IF EXISTS music_library_by_song")
except Exception as e:
    print(e)
        

### Close the session and cluster connection¶

In [36]:
session.shutdown()
cluster.shutdown()